In [2]:
#ML final project
#Yu Zhou, Che Wang

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
import numpy as np
import keras
from keras import backend as K
import matplotlib.image as mpimg
from glob import glob
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# A bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline 
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy import misc
"""datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')"""
#don't worry about data augmentation for now...
#data preprocessing is done in another file
print()

Using TensorFlow backend.


In [14]:

globPaths = ['data/c0small/*.jpg','data/c1small/*.jpg','data/c2small/*.jpg','data/c3small/*.jpg','data/c4small/*.jpg',
            'data/c5small/*.jpg','data/c6small/*.jpg','data/c7small/*.jpg',
            'data/c8small/*.jpg','data/c9small/*.jpg']
num_classes = 10
numTrain = 1500

X_train = np.empty((15000,64,64,3))
X_test = np.empty((4000,64,64,3))
y_train = np.empty((15000))
y_test = np.empty((4000))

i = 0
j = 0

for folderIndex in range(10):
    index = 0
    filelist = glob(globPaths[folderIndex])
    for filename in filelist:
        newImg = misc.imread(filename)
        if index<numTrain:
            X_train[i] = newImg
            y_train[i] = folderIndex
            i+=1
        else:
            X_test[j] = newImg
            y_test[j] = folderIndex
            j+=1
        index+=1
        if index%200==0:
            print("reading img,",folderIndex,index)
        
        
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
#then data normalization
X_train /= 255
X_test /= 255

print("y_train",y_train.shape[0])
print("y_test",y_test.shape[0])

Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)
print("data read and normalization finished!")



reading img, 0 200
reading img, 0 400
reading img, 0 600
reading img, 0 800
reading img, 0 1000
reading img, 0 1200
reading img, 0 1400
reading img, 0 1600
reading img, 0 1800
reading img, 1 200
reading img, 1 400
reading img, 1 600
reading img, 1 800
reading img, 1 1000
reading img, 1 1200
reading img, 1 1400
reading img, 1 1600
reading img, 1 1800
reading img, 2 200
reading img, 2 400
reading img, 2 600
reading img, 2 800
reading img, 2 1000
reading img, 2 1200
reading img, 2 1400
reading img, 2 1600
reading img, 2 1800
reading img, 3 200
reading img, 3 400
reading img, 3 600
reading img, 3 800
reading img, 3 1000
reading img, 3 1200
reading img, 3 1400
reading img, 3 1600
reading img, 3 1800
reading img, 4 200
reading img, 4 400
reading img, 4 600
reading img, 4 800
reading img, 4 1000
reading img, 4 1200
reading img, 4 1400
reading img, 4 1600
reading img, 4 1800
reading img, 5 200
reading img, 5 400
reading img, 5 600
reading img, 5 800
reading img, 5 1000
reading img, 5 1200
read

In [15]:
# input image dimensions
img_rows, img_cols = 64, 64
img_channels = 3
#the later are about training...

input_shape = (img_rows, img_cols, img_channels)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),padding= 'same',
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt, #keras.optimizers.Adadelta(0.01),
              metrics=['accuracy'])
print("model building and compiling finished!")


model building and compiling finished!


In [16]:
batch_size = 32 
nb_epoch = 5
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=nb_epoch,
          verbose=1,
          validation_data=(X_test, Y_test))

pred = model.predict(X_train,batch_size=10)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save('keras_cnn_model.h5')

Train on 15000 samples, validate on 4000 samples
Epoch 1/5
15000/15000 [==============================] - 674s - loss: 1.4187 - acc: 0.4813 - val_loss: 0.5449 - val_acc: 0.7990
Epoch 2/5
15000/15000 [==============================] - 642s - loss: 0.5981 - acc: 0.7644 - val_loss: 0.3223 - val_acc: 0.8510
Epoch 3/5
15000/15000 [==============================] - 632s - loss: 0.4121 - acc: 0.8193 - val_loss: 0.2509 - val_acc: 0.8725
Epoch 4/5
15000/15000 [==============================] - 661s - loss: 0.3517 - acc: 0.8394 - val_loss: 0.2426 - val_acc: 0.8732
Epoch 5/5
15000/15000 [==============================] - 627s - loss: 0.3106 - acc: 0.8536 - val_loss: 0.2199 - val_acc: 0.8808
Test loss: 0.219861876066
Test accuracy: 0.88075
